This aims to be a tutorial for beginners that already introduces cocepts that until
now are mostly used by experienced users.

We want to model a group of buildings, each with demands for electricity and space heating.
Each has its individual heat pump and is connected to the electricity grid.
Some of the buildings have extra features, such as demand for domestic hot water or a PV system.

First, we define an energy system. Note that solph expects time points for its index.
Time intervals are defined between the points in time.
If your points in time have a regular pattern, you can (at your option) infer the last interval.
Typically, it's better to explicitly give N+1 points in time to define N time intervals.

In [ ]:
import datetime
from oemof import solph

n_time_points = 25

first_timepoint = datetime.datetime(2023,2,1,6)
time_index = solph.create_time_index(2024, number=n_time_points, start=first_timepoint)

energy_system = solph.EnergySystem(
    timeindex=time_index,
    infer_last_interval=False,
)


In [ ]:
import numpy as np
import pandas as pd

temperatures = pd.read_csv(
    "input_data.csv",
    parse_dates=[0],
    index_col=0,
    date_format="%Y%m%d%H",
)

input_data = temperatures.reindex(time_index)

locations = ["house_1", "house_2"]
heat_up_to = [12, 15]
power_per_degree = [0.7, 1.1]

for location, temperature, power in zip(locations, heat_up_to, power_per_degree):
    input_data[f"heat_demand_{location}"] = np.maximum(temperature - input_data["TT_TU"], 0) * power

The energy system is modelled as a mathematcal graph.
Often, `Bus`es are used to model commodities.
These are then connected by `Converter`s and to storages.
Each of these nodes needs a unique label to be identified.
Edges are directed and identified by the Nodes they connect to.

Many users use strings as labels.
However, as energy systems become complex, keeping track of all the information can be hard.
In particular, manually managing string labels can be tedious.
Here, it comes handy that `Node`s (defined in oemof.network) accept every hashable type.

A custom string representation is advised as the defauls (including type names) can be very long.
But labels should be easy to understand.

In [ ]:
from typing import NamedTuple
from enum import IntEnum

# A frozenset is an immutable set.
# Set: Every sector present only once.
# Imutable:
#    - sectors cannot be changed
#    - makes the class hashable
class Label(NamedTuple):
    location: str
    sectors: frozenset[int]
    component: str

    def __str__(self):
        return f"{self.location}/{sum(int(s) for s in self.sectors)}/{self.component}"


For the sectors, we create an Enum.
Enums are very useful to make sure, only predefined values are used.
This, e.g. prevents typos staying unnoticed.
The IntEnum in the example is defined in a way that any combination of sectors gives a unique ID.
This is a bit C-Style low-level encoding but can be useful, e.g. when saving the info to a file.

In [ ]:

class Sectors(IntEnum):
    ELECTRICITY = 1
    HEAT = 2
    HYDROGEN = 4


The Label above needs a frozenset.
To freeze sets in the background, we create a factory function.

In [ ]:

def label(
    location: str,
    sectors: set[int],
    component: str,
) -> Label:
    return Label(location, frozenset(sectors),component)

We now populate the energy system. It consists of several houses and a grid.
Let's start with the grid, as every house should be able to connect to it.
(Note that some of the functionality does not rely on storing information in the label.
It could be placed elsewhere. However, it is convenient to have it there, as you will see soon.)

In [ ]:
location="grid"

b_el_grid = solph.Bus(label(location, {Sectors.ELECTRICITY}, "Bus"))
energy_system.add(b_el_grid)

energy_system.add(
    solph.components.Source(
        label(location, {Sectors.ELECTRICITY}, "grid_connection"),
        outputs={b_el_grid: solph.Flow(variable_costs=0.4)},
        # custom_attributes={"sectors": {Sectors.ELECTRICITY}}
    )
)


The houses start with an identical base: One Bus for electricity and one for heat.
Note that the busses are just called "Bus".

In [ ]:

locations = ["house_1", "house_2"]

for number, location in enumerate(locations):
    b_el = solph.Bus(
        label(location, {Sectors.ELECTRICITY}, "Bus"),
        inputs={b_el_grid: solph.Flow()},
        outputs={b_el_grid: solph.Flow()},
    )
    b_heat = solph.Bus(label(location, {Sectors.HEAT}, "Bus"))

    energy_system.add(b_el, b_heat)

    energy_system.add(
        solph.components.Sink(
            label(location, {Sectors.HEAT}, "Demand"),
            inputs={
                b_heat: solph.Flow(nominal_value=1, fix=input_data[f"heat_demand_{location}"])
            },
        )
    )

    cop = 3
    energy_system.add(
        solph.components.Converter(
            label(location, {Sectors.ELECTRICITY, Sectors.HEAT}, "heat_pump"),
            inputs={b_el: solph.Flow()},
            outputs={b_heat: solph.Flow()},
            conversion_factors={b_el: 1 / cop},
        )
    )


Now, we can add custom features to the houses.
House 1 receives additional demand for domestic hot water in just one hour.
It's not in the unput data, so we create something manually.
(This is possible, but not advised. It's always good to have a time index.)

Note that we access the electricity bus using its label.
(The API is experimental, the way to do this might be changed with the next release of oemof.network.)

In [ ]:
location = "house_1"

b_dhw = solph.Bus(label(location, {Sectors.HEAT}, "Bus_DHW"))
energy_system.add(b_dhw)

energy_system.add(
    solph.components.Sink(
        label(location, {Sectors.HEAT}, "DHW_Demand"),
        inputs={b_dhw: solph.Flow(
            nominal_value=1,
            fix=6*[0] + 1*[12] + 18*[0],
        )},
    )
)

b_el = energy_system.node[label(location, {Sectors.ELECTRICITY}, "Bus")]
energy_system.add(
    solph.components.Converter(
        label(location, {Sectors.ELECTRICITY, Sectors.HEAT}, "flow_heater"),
        inputs={b_el: solph.Flow()},
        outputs={b_dhw: solph.Flow()},
    )
)

House 2 gets a PV system.
There are two common ways to model it:
* Seing PV generation as fix Flow, making an excess `Sink` obligatory.
* Setting the PV series as the maximum possible Flow.

In [ ]:
energy_system.add(
    solph.components.Source(
        label("house_2", {Sectors.ELECTRICITY}, "PV"),
        outputs={
            energy_system.node[label("house_2", {Sectors.ELECTRICITY}, "Bus")]:
                solph.Flow(nominal_value=50, max=input_data["PV"]),
        },
    )
)

Before we optimise the proble, we visually check the graph.

In [ ]:
import networkx as nx
from oemof.network.graph import create_nx_graph

graph = create_nx_graph(energy_system)

nx.draw(graph, with_labels=True, font_size=8)

In [ ]:
model = solph.Model(energy_system)
model.solve(solver="cbc", solve_kwargs={"tee": False})
results = solph.processing.results(model)

In [ ]:
import pandas as pd

flows_to_heat = pd.DataFrame({
    f"{k[0].label.location}-{k[0].label.component}": v["sequences"]["flow"]
    for k, v in results.items()
    if (
        isinstance(k[1], solph.Bus)
        and Sectors.HEAT in k[1].label.sectors
    )
})

heat_demand = pd.DataFrame({
    f"{k[0].label.location}-{k[1].label.component}": v["sequences"]["flow"]
    for k, v in results.items()
    if isinstance(k[1], solph.components.Sink)
})

heat_flows = pd.concat(
    [flows_to_heat, heat_demand],
    axis=1,
)

In [ ]:
import matplotlib.pyplot as plt

heat_flows.plot(drawstyle="steps-post")

plt.show()

In [ ]:
electricity_sources = pd.DataFrame({
    f"{k[0].label.location}-{k[0].label.component}": v["sequences"]["flow"]
    for k, v in results.items()
    if isinstance(k[0], solph.components.Source)
    and Sectors.ELECTRICITY in k[0].label.sectors
})

previous_sources = np.zeros(len(electricity_sources))
for source in electricity_sources:
    current_source = previous_sources + electricity_sources[source]
    plt.fill_between(
        electricity_sources.index,
        previous_sources,
        current_source,
        label=source,
        step="post",
    )
    previous_sources = current_source

plt.legend()
plt.show()